# Double Pendulum Simulation
### by Ganden Schaffner and Ian McDougall

In [ ]:
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
# import ipywidgets as widg

%matplotlib notebook

def derivs(state, t):
    dxdt = np.zeros_like(state) # initialize dervative array (values correspond to the time derivatives of initial state array)

    dxdt[0] = state[1] # derivative of angle1

    th_diff_init = state[2] - state[0] # difference of angle1 and angle2

    denom1 = (M1 + M2)*L1 - M2*L1*cos(th_diff_init)*cos(th_diff_init) # denominator of dxdt[2]
    dxdt[1] = (M2*L1*state[1]*state[1]*sin(th_diff_init)*cos(th_diff_init) +
               M2*G*sin(state[2])*cos(th_diff_init) +
               M2*L2*state[3]*state[3]*sin(th_diff_init) -
               (M1 + M2)*G*sin(state[0]))/denom1 # time derivative of angular velocity 1

    dxdt[2] = state[3] # time derivative of angle2

    denom2 = (L2/L1)*denom1 # denominator of dxdt[3]
    dxdt[3] = (-M2*L2*state[3]*state[3]*sin(th_diff_init)*cos(th_diff_init) +
               (M1 + M2)*G*sin(state[0])*cos(th_diff_init) -
               (M1 + M2)*L1*state[1]*state[1]*sin(th_diff_init) -
               (M1 + M2)*G*sin(state[2]))/denom2 # time derivative of angular velocity 2

    return dxdt # return the array of time derivatives

# Initializes the plot
def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

# Animates the line (this is called sequentially)
def animate(i):
    newx = [0, x1[i], x2[i]]
    newy = [0, y1[i], y2[i]]

    line.set_data(newx, newy)
    time_text.set_text(time_template % (i*dt)) # ??????????
    return line, time_text

# Set default initial conditions
th1 = 120.0  # th1 and th2 are the initial angles (degrees)
th2 = 0.0
w1 = -10.0     # w10 and w20 are the initial angular velocities (degrees per second)
w2 = 0.0

G =  9.81     # acceleration due to gravity (in m/s^2)
L1 = 1.0      # length of pendulum (1 in m)
L2 = 1.0      # length of pendulum 2 (in m)
M1 = 1.0      # mass of pendulum 1 (in kg)
M2 = 1.0      # mass of pendulum 2 (in kg)
total_t = 20  # length of the animation

# Set these variables so they may be used as globals
line, time_text, fig, ax, y_len, x1, x2, y1, y2 = 0, 0, 0, 0, 0, 0, 0, 0, 0

def plot_pendulums():
    global line, time_text, fig, ax, y_len, x1, x2, y1, y2
    
    dt = 0.05 # Interval between successive pendulum states to calculate
    t = np.arange(0.0, total_t, dt)

    # Calculate the pendulum's position throughout the animation
    state = np.radians([th1, w1, th2, w2])

    y = integrate.odeint(derivs, state, t)

    x1 = L1*sin(y[:, 0])
    y1 = -L1*cos(y[:, 0])
    x2 = L2*sin(y[:, 2]) + x1
    y2 = -L2*cos(y[:, 2]) + y1
    
    # Plot!
    # Set up the figure, axis, and line using the default initial conditions
    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-1*(L1+L2), L1+L2), ylim=(-1*(L1+L2), L1+L2)) # 111 means 1x1 grid, first subplot

    line, = ax.plot([], [], 'o-', lw=2) # o- sets a solid line and circular markers
    line_template = 'time = %.1fs' # format as a float to the tenths place
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    y_len = len(y)

plot_pendulums()
# Call the animator. Only re-draw the parts that have changed (via blit=True)
anim = animation.FuncAnimation(fig, animate, np.arange(1, y_len), interval=25, blit=True, init_func=init)
plt.show()